In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torch.utils.data import DataLoader


In [ ]:
class VGG16(nn.Module):

  """
  Takes input image of size 224x224
     Parameters:

        in_channels   : no of input channels in the image 1 if input image is in grayscal 3 if input image is in color (defaults to 3)
        num_classes   : no of classes in your classification setup (defaults to 1000)
  """

  # defining the architecture as list
  architecture=[64,64,'M',128,128,'M',256,256,256,'M',512,512,512,'M',512,512,512,'M']

  def __init__(self,in_channels=3,num_classes=1000):

    super(VGG16,self).__init__()
    self.in_channels=in_channels
    self.num_classes=num_classes
    # creating convolutional layers
    self.conv_layers=self.create_conv_layers(self.architecture)
    # creating FC layers
    self.fc_layers=nn.Sequential(nn.Linear(in_features=7*7*512,out_features=4096),
                                 nn.ReLU(),
                                 nn.Dropout(p=0.5),
                                 nn.Linear(in_features=4096,out_features=4096),
                                 nn.ReLU(),
                                 nn.Dropout(p=0.5),
                                 nn.Linear(in_features=4096,out_features=self.num_classes))


  def create_conv_layers(self,architecture):

    # defining empty list to store the layers of vgg16
    layers=[]
    in_channels=self.in_channels
    # iterating through the architecture list to create layers
    for i in architecture:

      # if the i is an int then it is a conv layer having i no of kernels
      if type(i)==int:
        out_channels=i
        # creating Conv layer with batchnorm and relu and appending it to the layers list
        layers+=[nn.Conv2d(in_channels=in_channels,out_channels=out_channels,kernel_size=3,stride=1,padding=1),
                      nn.BatchNorm2d(num_features=i),
                      nn.ReLU()]
        # updating the in_channel for the next layer (out_channels in the current layer ====> in_channel for next layer)
        in_channels=i

      # if i is M then it is a Max Pool layer
      elif i=="M":
        layers.append(nn.MaxPool2d(kernel_size=2,stride=2))

    # creating the model using the sequential API and returning the model
    return nn.Sequential(*layers)


  def forward(self,x):
    # passing images through the convolutional layers
    x=self.conv_layers(x)
    # passing images through the FC layers
    x = x.view(x.size(0), -1)
    x=self.fc_layers(x)
    return x


  

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader

# Define transforms to normalize the data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224))  # Resize to match VGG input size
])

# Load CIFAR training and test datasets
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = DataLoader(testset, batch_size=64,
                                         shuffle=False, num_workers=2)

# Define the device for training
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Initialize the VGG model
model = VGG16(num_classes=10).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training the model
for epoch in range(5):  # You can increase the number of epochs
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        if i % 200 == 199:  # Print every 200 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

print('Finished Training')

# Evaluate the model
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))


100%|██████████| 170498071/170498071 [00:03<00:00, 42857247.21it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across th